In [5]:
import numpy as np
import pandas as pd
import os

from sklearn.cluster import DBSCAN, KMeans
from scipy.spatial.distance import cdist

In [6]:
# Excel 파일 경로
file_path = "./Nutri_food_DB.xlsx"

# Excel 파일 읽기
orifing_df = pd.read_excel(file_path)

In [7]:
orifing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7704 entries, 0 to 7703
Data columns (total 100 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   NO                       7704 non-null   int64 
 1   SAMPLE_ID                7704 non-null   object
 2   식품코드                     7704 non-null   object
 3   DB군                      7704 non-null   object
 4   상용제품                     7704 non-null   object
 5   식품명                      7704 non-null   object
 6   연도                       7704 non-null   int64 
 7   지역 / 제조사                 7704 non-null   object
 8   채취시기                     7704 non-null   object
 9   식품대분류                    7704 non-null   object
 10  식품상세분류                   7704 non-null   object
 11  1회제공량                    7704 non-null   int64 
 12  내용량_단위                   7704 non-null   object
 13  총내용량(g)                  7704 non-null   object
 14  총내용량(mL)                 7704 non-null 

In [8]:
orifing_df = orifing_df.sort_values('연도', ascending=False).drop_duplicates('식품명')

In [9]:
orifing_df.columns

Index(['NO', 'SAMPLE_ID', '식품코드', 'DB군', '상용제품', '식품명', '연도', '지역 / 제조사',
       '채취시기', '식품대분류', '식품상세분류', '1회제공량', '내용량_단위', '총내용량(g)', '총내용량(mL)',
       '에너지(㎉)', '수분(g)', '단백질(g)', '지방(g)', '탄수화물(g)', '총당류(g)', '자당(g)',
       '포도당(g)', '과당(g)', '유당(g)', '맥아당(g)', '총 식이섬유(g)', '칼슘(㎎)', '철(㎎)',
       '철(㎍)', '마그네슘(㎎)', '인(㎎)', '칼륨(㎎)', '나트륨(㎎)', '아연(㎎)', '구리(㎎)', '망간(㎎)',
       '셀레늄(㎍)', '레티놀(㎍)', '베타카로틴(㎍)', '비타민 D3(㎍)', '토코페롤(㎎)', '토코트리에놀(㎎)',
       '비타민 B1(㎎)', '비타민 B2(㎎)', '나이아신(㎎)', '엽산(DFE)(㎍)', '비타민 B12(㎍)',
       '비타민 C(㎎)', '총 아미노산(㎎)', '이소류신(㎎)', '류신(㎎)', '라이신(㎎)', '메티오닌(㎎)',
       '페닐알라닌(㎎)', '트레오닌(㎎)', '발린(㎎)', '히스티딘(㎎)', '아르기닌(㎎)', '티로신(㎎)',
       '시스테인(㎎)', '알라닌(㎎)', '아스파르트산(㎎)', '글루탐산(㎎)', '글리신(㎎)', '프롤린(㎎)',
       '세린(㎎)', '콜레스테롤(㎎)', '총 포화 지방산(g)', '부티르산(4:0)(g)', '카프로산(6:0)(g)',
       '카프릴산(8:0)(g)', '카프르산(10:0)(g)', '라우르산(12:0)(g)', '미리스트산(14:0)(g)',
       '팔미트산(16:0)(g)', '스테아르산(18:0)(g)', '아라키드산(20:0)(g)', '미리스톨레산(14:1)(g)',
       '팔미톨레산(16:1)(g)', '올

In [10]:
# 데이터프레임 출력
df = orifing_df[orifing_df["지역 / 제조사"] == "전국(대표)"][["식품명", "1회제공량", "식품대분류", "에너지(㎉)", "탄수화물(g)", "단백질(g)", "지방(g)", "총당류(g)", "나트륨(㎎)"]]

In [11]:
df

,식품명,1회제공량,식품대분류,에너지(㎉),탄수화물(g),단백질(g),지방(g),총당류(g),나트륨(㎎)
7063,잔멸치고추볶음,50,볶음류,114.05,2.86,16.64,4,1.88,1030.52
7074,멸치풋고추볶음,50,볶음류,94.83,6.65,10.31,3,4.51,672.44
7073,팔보채,220,볶음류,196.98,7.53,27.33,6.39,4.59,1883.99
7072,콩나물잡채,230,볶음류,153.12,28.86,4.43,2.22,21.56,1422.94
7071,채소떡볶이,200,볶음류,515.29,114.11,6.24,3.76,3.94,537.97
...,...,...,...,...,...,...,...,...,...
157,양송이버섯볶음,150,볶음류,136,11,4.5,8.3,1.2,515.17
156,순대볶음,400,볶음류,582,76.7,20.7,21.3,5.4,1500.35
155,소세지볶음,200,볶음류,471,31.8,18.8,29.9,8.9,1436.06
153,머위나물볶음,150,볶음류,108,7.6,3.5,7,1.8,718.63


In [12]:
df["에너지(㎉)"] = df["에너지(㎉)"].astype("float")
df["탄수화물(g)"] = df["탄수화물(g)"].astype("float")
df["단백질(g)"] = df["단백질(g)"].astype("float")
df["지방(g)"] = df["지방(g)"].astype("float")
df["총당류(g)"] = df["총당류(g)"].astype("float")
df['나트륨(g)'] = df['나트륨(㎎)'].astype("float") / 1000

In [13]:
df = df.drop(['나트륨(㎎)'], axis=1)

In [14]:
main_manu_list = []

with open(os.path.join("./fin_main_menu.txt"), 'r', encoding="utf-8") as f:
    for line in f:
        main_manu_list.append(line.replace("\n",""))

In [15]:
df = df[df['식품명'].isin(main_manu_list)]

In [16]:
df.식품대분류.value_counts()

밥류          42
찌개 및 전골류    42
구이류         31
면 및 만두류     22
볶음류         21
국 및 탕류      17
튀김류         16
찜류          15
조림류         11
회류           1
Name: 식품대분류, dtype: int64

In [17]:
dflist = []

for class_name in df.식품대분류.value_counts().index:
    dflist.append(df[df.식품대분류 == class_name])

# DBSCAN

In [18]:
dflist

[           식품명  1회제공량 식품대분류   에너지(㎉)  탄수화물(g)  단백질(g)  지방(g)  총당류(g)   나트륨(g)
 7018     하이라이스    360    밥류   401.93    76.84   14.47   4.08    0.00  0.88409
 7017   표고버섯볶음밥    300    밥류   551.73    95.37   10.09  14.43    0.00  0.89657
 7014     콩나물국밥    780    밥류   403.26    85.22   11.34   1.89    0.00  1.33902
 7013     카레라이스    480    밥류   518.40    99.53   14.94   6.73    0.78  1.00705
 7012      치즈김밥    270    밥류   476.98    59.67   16.86  18.99    0.57  0.45554
 7011     채소볶음밥    260    밥류   474.03    74.73   12.26  14.01    0.76  1.06034
 7010      채소김밥    280    밥류   442.09    74.61   12.89  10.23    0.40  0.86501
 7009     참치볶음밥    230    밥류   412.97    75.68   10.63   7.52    0.27  1.45092
 7008  참치마요삼각김밥    200    밥류   398.70    50.89   11.86  16.41    0.00  0.46022
 7007      참치김밥    250    밥류   435.04    50.65   17.50  18.05    1.77  0.83743
 7004       짬뽕밥    900    밥류   647.70   104.86   28.81  12.56   12.04  2.10568
 7003       짜장밥    470    밥류   572.62    73.17   20.

In [490]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 7073 to 275
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   식품명      218 non-null    object 
 1   1회제공량    218 non-null    int64  
 2   식품대분류    218 non-null    object 
 3   에너지(㎉)   218 non-null    float64
 4   탄수화물(g)  218 non-null    float64
 5   단백질(g)   218 non-null    float64
 6   지방(g)    218 non-null    float64
 7   총당류(g)   218 non-null    float64
 8   나트륨(g)   218 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 17.0+ KB


In [498]:
df_copy = df.copy()

In [499]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 7073 to 275
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   식품명      218 non-null    object 
 1   1회제공량    218 non-null    int64  
 2   식품대분류    218 non-null    object 
 3   에너지(㎉)   218 non-null    float64
 4   탄수화물(g)  218 non-null    float64
 5   단백질(g)   218 non-null    float64
 6   지방(g)    218 non-null    float64
 7   총당류(g)   218 non-null    float64
 8   나트륨(g)   218 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 17.0+ KB


In [78]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN

cluster_num = 0
start_num = 0
end_num = 218

outliers = []
replist = []

cluster_items = []

center_list = []

for df_copy in dflist[start_num:end_num]:
    # 데이터프레임의 X, Y, Name 열을 특성으로 사용
    data = df_copy[["에너지(㎉)","탄수화물(g)", "단백질(g)", "지방(g)", "총당류(g)", "나트륨(g)"]].values
    item_names = df_copy["식품명"].values

    # DBSCAN 알고리즘 적용
    epsilon = 90
    minPts = 10

    dbscan = DBSCAN(eps=epsilon, min_samples=minPts)
    dbscan.fit(data)

    # 각 군집의 중심 좌표
    cluster_centers = dbscan.components_
    labels = dbscan.labels_
    unique_labels = np.unique(labels)
    n_clusters = len(unique_labels) - (1 if -1 in labels else 0)
#     print('@@@@@',cluster_centers)
    
    # 각 군집의 대표 아이템 찾기
    representative_items = []
    for i in range(n_clusters):
        cluster_i = data[labels == i]
        center_i = cluster_centers[i]
        distances = cdist(cluster_i, [center_i], metric='euclidean')
        closest_index = np.argmin(distances)
        representative_item = cluster_i[closest_index]
        representative_items.append(representative_item)

    # 대표 아이템 출력
    
    for i, item in enumerate(representative_items):
        cluster_num += 1
        cluster_name = 'Cluster {}'.format(cluster_num)
        item_name = item_names[np.where((data == item).all(axis=1))][0]
        print(cluster_name, ":", item_name, item)
        replist.append(str(item_name))
        center_i = list(center_i)
        center_i.append(cluster_num)
        center_list.append(center_i)
        

    print("="*50)
    # 군집 결과를 데이터프레임에 추가
    df_copy['Cluster'] = dbscan.labels_
    df_copy['Cluster Name'] = df_copy['Cluster'].map(lambda x: 'Cluster '+str(x+1) if x >= 0 else 'Outlier')
    # 군집화 결과 확인
#     print(df_copy)
#     center_list.append(center_i)

    for num in df_copy.Cluster.value_counts().index:
        if num > -2: 
            if num == -1:
                outliers.extend(list(df_copy[df_copy.Cluster==num].식품명))
            else: 
                cluster_items.append(list(df_copy[df_copy.Cluster==num].식품명))
                print(num+1, list(df_copy[df_copy.Cluster==num].식품명))
    print("="*50)
    print()
    print()

print("outliers", outliers)

for outlier in outliers:
    outlier_list = list(df[df["식품명"]==outlier][["에너지(㎉)","탄수화물(g)","단백질(g)","지방(g)","총당류(g)","나트륨(g)"]].reset_index(drop=True).iloc[0,:])
    outlier_list.append(0)
    center_list.append(outlier_list)

Cluster 1 : 하이라이스 [401.93     76.84     14.47      4.08      0.        0.88409]
1 ['하이라이스', '표고버섯볶음밥', '콩나물국밥', '카레라이스', '치즈김밥', '채소볶음밥', '채소김밥', '참치볶음밥', '참치마요삼각김밥', '참치김밥', '짬뽕밥', '짜장밥', '육회비빔밥', '유부초밥', '오징어덮밥', '오므라이스', '영양돌솥밥', '알밥', '계란볶음밥', '고추김밥', '순대국밥', '쇠고기주먹밥', '쇠고기볶음밥', '쇠고기김밥', '생선모듬초밥', '새우볶음밥', '비빔밥', '불고기덮밥', '볶음밥', '모듬회덮밥', '돌솥비빔밥', '돈까스김밥', '닭고기덮밥', '날치알김밥', '낙지덮밥', '김치볶음밥', '김치김밥', '김밥']


Cluster 2 : 호박고추장찌개 [115.04     16.69      5.83      2.77      1.58      0.59073]
1 ['호박고추장찌개', '햄 김치찌개', '해물매운탕', '콩비지찌개', '콩나물된장찌개', '추어탕', '청국장찌개', '참치김치찌개', '조개된장찌개', '오징어찌개', '오리백숙', '어묵김치찌개', '알탕', '순두부찌개', '순두부김치찌개', '쇠고기전골', '쇠고기된장찌개', '쇠고기 샤브샤브', '버섯전골', '버섯샤브샤브', '두부애호박된장찌개', '두부쇠고기고추장찌개', '두부된장찌개', '두부고추장찌개', '된장찌개', '돼지고기찌개', '돼지고기김치찌개', '동태찌개', '대구 매운탕', '닭볶음탕', '달래된장찌개', '냉이된장찌개', '낙지전골', '꽃게탕', '꽁치김치찌개', '김치전골', '갈치찌개']


Cluster 3 : 함박스테이크 [3.4737e+02 1.4380e+01 2.4200e+01 2.1450e+01 1.1000e-01 3.9528e-01]
1 ['함박스테이크', '닭발구이', '닭고기대파꼬치구이', '굴비구이', '갈치구이', '가자미구이', 

C:\Users\acorn\AppData\Local\Temp\ipykernel_10316\3257955275.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['Cluster'] = dbscan.labels_
C:\Users\acorn\AppData\Local\Temp\ipykernel_10316\3257955275.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['Cluster Name'] = df_copy['Cluster'].map(lambda x: 'Cluster '+str(x+1) if x >= 0 else 'Outlier')
C:\Users\acorn\AppData\Local\Temp\ipykernel_10316\3257955275.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [69]:
center_list1 = pd.DataFrame(center_list)
center_list1.columns = ["에너지(㎉)","탄수화물(g)","단백질(g)","지방(g)","총당류(g)","나트륨(g)","Cluster"]

In [71]:
center_list1.Cluster.astype("str")

0     1
1     2
2     3
3     4
4     5
     ..
78    0
79    0
80    0
81    0
82    0
Name: Cluster, Length: 83, dtype: object

In [101]:
remain={'에너지(㎉)' : [-371.53],
 '탄수화물(g)' :[ -156.8],
 '단백질(g)' : [-86.67],
 '지방(g)' : [-19.7],
 '총당류(g)' : [-13.39],
 '나트륨(g)' : [-4.04]}

remain = pd.DataFrame(remain)

In [90]:
remain

,에너지(kcal),탄수화물(g),단백질(g),지방(g),총당류(g),나트륨(g)
0,-371.53,-156.8,-86.67,-19.7,-13.39,-4.04


In [114]:
result2={'Cluster':[],'나트륨(g)' : [],'단백질(g)' : [], '에너지(㎉)' : [],'지방(g)' : [], '총당류(g)' : [],'총합':[], '탄수화물(g)' :[],
 
 }

result2= pd.DataFrame(result2)
result2

,Cluster,나트륨(g),단백질(g),에너지(㎉),지방(g),총당류(g),총합,탄수화물(g)


In [118]:
center_list1.iloc[0]+remain.iloc[0]

Cluster         NaN
나트륨(g)     -3.15591
단백질(g)    -72.20000
에너지(㎉)     30.40000
지방(g)     -15.62000
총당류(g)    -13.39000
총합              NaN
탄수화물(g)   -79.96000
Name: 0, dtype: float64

In [120]:
result2= pd.DataFrame(center_list1.iloc[0]+remain.iloc[0]).T

,Cluster,나트륨(g),단백질(g),에너지(㎉),지방(g),총당류(g),총합,탄수화물(g)
0,NaN,-3.15591,-72.2,30.4,-15.62,-13.39,NaN,-79.96


In [124]:
selected_columns = ['식품명', '에너지(㎉)', '단백질(g)', '지방(g)', '탄수화물(g)']

In [125]:
for columns_idx, columns in enumerate(selected_columns[1:]):
    print(columns_idx, columns)

0 에너지(㎉)
1 단백질(g)
2 지방(g)
3 탄수화물(g)


In [75]:
# center_list1.margin()
center_list1["총합"] = np.sum(center_list1, axis=1)

In [76]:
center_list1

,에너지(㎉),탄수화물(g),단백질(g),지방(g),총당류(g),나트륨(g),Cluster,총합
0,401.93,76.84,14.47,4.08,0.00,0.88409,1,499.20409
1,115.04,16.69,5.83,2.77,1.58,0.59073,2,144.50073
2,347.37,14.38,24.20,21.45,0.11,0.39528,3,410.90528
3,464.46,66.06,22.30,12.34,12.52,1.44606,4,583.12606
4,276.31,8.69,31.46,12.86,4.77,0.49250,5,339.58250
...,...,...,...,...,...,...,...,...
78,65.31,9.09,5.76,0.66,2.20,0.71057,0,83.73057
79,124.64,2.61,15.88,5.63,2.03,0.44101,0,151.23101
80,112.38,4.87,12.46,4.78,0.22,0.84221,0,135.55221
81,331.41,19.42,27.23,16.09,15.78,0.47237,0,410.40237


In [494]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 7164 to 7164
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   식품명           1 non-null      object 
 1   1회제공량         1 non-null      int64  
 2   식품대분류         1 non-null      object 
 3   에너지(㎉)        1 non-null      float64
 4   탄수화물(g)       1 non-null      float64
 5   단백질(g)        1 non-null      float64
 6   지방(g)         1 non-null      float64
 7   총당류(g)        1 non-null      float64
 8   나트륨(g)        1 non-null      float64
 9   Cluster       1 non-null      int64  
 10  Cluster Name  1 non-null      object 
dtypes: float64(6), int64(2), object(3)
memory usage: 96.0+ bytes


In [331]:
outliers50_10 = ['제육덮밥', '잡채밥', '쇠머리국밥', '불고기덮밥', '돼지머리국밥', '김치볶음밥', '장어탕', '오리탕', '오리백숙', '부대찌개', '닭볶음탕', '내장탕', '감자탕', '닭발구이', '꽁치구이', '고등어석쇠구이', '고등어구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '돼지불고기', '회냉면', '콩국수', '쫄면', '잔치국수', '우동(일식)', '수제비', '볶음우동', '미트볼 토마토 스파게티', '물냉면', '메밀국수', '막국수', '라면', '달걀라면', '간짜장', '팔보채', '채소떡볶이', '참치 김치볶음', '제육볶음', '유산슬', '쇠고기볶음', '햄채소볶음', '문어고추장볶음', '돼지고기채소볶음', '돼지고기김치볶음', '닭볶음', '닭발볶음', '낙지볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '등심돈까스', '닭강정', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '달걀찜(새우젓)', '닭찜', '해물콩나물찜', '족발', '순대', '쇠고기수육', '쇠고기사태찜', '돼지고기 수육', '돼지갈비찜', '꽁치조림', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outliers50_10)

80

In [335]:
outliers60_9 = ['제육덮밥', '잡채밥', '쇠머리국밥', '돼지머리국밥', '장어탕', '오리탕', '오리백숙', '부대찌개', '내장탕', '감자탕', '꽁치구이', '고등어석쇠구이', '고등어구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '콩국수', '쫄면', '잔치국수', '우동(일식)', '수제비', '볶음우동', '미트볼 토마토 스파게티', '물냉면', '메밀국수', '막국수', '간짜장', '채소떡볶이', '제육볶음', '쇠고기볶음', '햄채소볶음', '돼지고기채소볶음', '돼지고기김치볶음', '닭볶음', '닭발볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '닭강정', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '닭찜', '족발', '순대', '쇠고기수육', '쇠고기사태찜', '돼지고기 수육', '돼지갈비찜', '꽁치조림', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outliers60_9)   

65

In [340]:
outliers70_9 = ['제육덮밥', '잡채밥', '쇠머리국밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '내장탕', '감자탕', '꽁치구이', '고등어석쇠구이', '고등어구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '쫄면', '잔치국수', '우동(일식)', '수제비', '볶음우동', '미트볼 토마토 스파게티', '물냉면', '메밀국수', '막국수', '채소떡볶이', '제육볶음', '쇠고기볶음', '햄채소볶음', '돼지고기채소볶음', '돼지고기김치볶음', '닭볶음', '닭발볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '족발', '순대', '쇠고기수육', '쇠고기사태찜', '돼지고기 수육', '돼지갈비찜', '꽁치조림', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outliers70_9)

60

In [345]:
outliers80_10 = ['제육덮밥', '잡채밥', '쇠머리국밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '내장탕', '고등어구이', '닭구이', '돼지갈비구이', '오리고기구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '쫄면', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '채소떡볶이', '제육볶음', '쇠고기볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '족발', '순대', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '꽁치조림', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outliers80_10)

45

In [349]:
outliers90_10 = ['제육덮밥', '잡채밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '고등어구이', '닭구이', '돼지갈비구이', '오리고기구이', '삼겹살구이', '회냉면', '우동(일식)', '미트볼 토마토 스파게티', '막국수', '채소떡볶이', '쇠고기볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '족발', '순대', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '꽁치조림', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outliers90_10)

39

In [353]:
outliers100_10 = ['제육덮밥', '잡채밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '고등어구이', '닭구이', '돼지갈비구이', '삼겹살구이', '우동(일식)', '미트볼 토마토 스파게티', '채소떡볶이', '쇠고기볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '깐풍기', '새우탕수육', '치즈돈가스', '족발', '순대', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '꽁치조림', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outliers100_10)

35

In [358]:
outliers110_10 = ['제육덮밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '삼겹살구이', '우동(일식)', '미트볼 토마토 스파게티', '채소떡볶이', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '새우탕수육', '족발', '순대', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outliers110_10)

26

In [362]:
outliers120_10 = ['제육덮밥', '돼지머리국밥', '오리탕', '부대찌개', '삼겹살구이', '우동(일식)', '미트볼 토마토 스파게티', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '새우탕수육', '족발', '순대', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '닭고추장조림', '육회']
len(outliers120_10)

22

In [366]:
outliers130_9 = ['제육덮밥', '돼지머리국밥', '오리탕', '부대찌개', '삼겹살구이', '우동(일식)', '미트볼 토마토 스파게티', '삼계탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '족발', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '닭고추장조림', '육회']
len(outliers130_9)

19

In [370]:
outliers140_9 = ['제육덮밥', '돼지머리국밥', '오리탕', '부대찌개', '우동(일식)', '미트볼 토마토 스파게티', '삼계탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '족발', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '육회']
len(outliers140_9)

17

In [374]:
outliers150_9 = ['제육덮밥', '돼지머리국밥', '오리탕', '부대찌개', '우동(일식)', '미트볼 토마토 스파게티', '삼계탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '족발', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '육회']
len(outliers150_9)

17

In [378]:
outliers175_9 = ['돼지머리국밥', '오리탕', '부대찌개', '우동(일식)', '삼계탕', '순댓국', '사골만두국', '떡만두국', '떡국', '족발', '돼지고기 수육', '돼지갈비찜', '육회']
len(outliers175_9)

13

In [382]:
outliers200_9 = ['돼지머리국밥', '오리탕', '부대찌개', '우동(일식)', '삼계탕', '돼지고기 수육', '육회']
len(outliers200_9)

7

In [438]:
outlier75_9_7 = ['제육덮밥', '잡채밥', '쇠머리국밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '내장탕', '감자탕', '꽁치구이', '고등어석쇠구이', '고등어구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '콩국수', '쫄면', '잔치국수', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '채소떡볶이', '참치 김치볶음', '제육볶음', '쇠고기볶음', '햄채소볶음', '문어고추장볶음', '돼지고기채소볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '닭찜', '족발', '순대', '쇠고기수육', '쇠고기사태찜', '쇠갈비찜', '돼지고기 수육', '돼지갈비찜', '꽁치조림', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outlier75_9_7)

60

In [434]:
outlier100_9_7 = ['제육덮밥', '잡채밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '꽁치구이', '고등어석쇠구이', '고등어구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '잔치국수', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '채소떡볶이', '제육볶음', '쇠고기볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '깐풍기', '새우탕수육', '치즈돈가스', '족발', '순대', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '꽁치조림', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outlier100_9_7)

47

In [430]:
outlier125_9_7 = ['제육덮밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '꽁치구이', '고등어석쇠구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '새우탕수육', '족발', '순대', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '닭고추장조림', '닭조림', '미트볼조림', '육회']
len(outlier125_9_7)

36

In [20]:
new_outlier = ['제육덮밥', '잡채밥', '쇠머리국밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '내장탕', '감자탕', '꽁치구이', '고등어석쇠구이', '고등어구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '콩국수', '쫄면', '잔치국수', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '간짜장', '채소떡볶이', '참치 김치볶음', '제육볶음', '쇠고기볶음', '햄채소볶음', '돼지고기채소볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '등심돈까스', '닭강정', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '달걀찜(우유)', '꼬막찜', '가오리찜', '달걀찜(새우젓)', '닭찜', '해물콩나물찜', '코다리찜', '족발', '아귀찜', '순대', '쇠고기수육', '쇠고기사태찜', '쇠갈비찜', '돼지고기 수육', '돼지갈비찜', '고등어조림', '건갈치조림', '갈치조림', '꽁치조림', '닭고추장조림', '닭조림', '코다리조림', '양미리조림', '삼치조림', '병어조림', '미트볼조림', '육회']
len(new_outlier)

76

In [21]:
df_goo10 = df_copy[df_copy['식품명'].isin(new_outlier)]

In [410]:
outlier75_5_10 = ['제육덮밥', '잡채밥', '쇠머리국밥', '돼지머리국밥', '장어탕', '오리탕', '오리백숙', '부대찌개', '내장탕', '감자탕', '꽁치구이', '고등어석쇠구이', '고등어구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '콩국수', '칼국수', '쫄면', '짬뽕', '짜장면', '쟁반국수', '잔치국수', '우동(일식)', '수제비', '비빔냉면', '비빔국수', '볶음우동', '바지락칼국수', '미트볼 토마토 스파게티', '물냉면', '메밀국수', '막국수', '라면', '떡라면', '달걀라면', '간짜장', '채소떡볶이', '참치 김치볶음', '제육볶음', '쇠고기볶음', '햄채소볶음', '문어고추장볶음', '돼지고기채소볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '등심돈까스', '닭날개튀김', '닭강정', '깐풍기', '고등어튀김', '간장양념치킨', '간장양념닭다리튀김', '새우탕수육', '후라이드치킨', '탕수육', '치킨까스', '치즈돈가스', '양념치킨', '안심돈가스', '생선까스', '등심돈가스', '달걀찜(우유)', '꼬막찜', '가오리찜', '달걀찜(새우젓)', '닭찜', '해물콩나물찜', '코다리찜', '족발', '아귀찜', '순대', '쇠고기수육', '쇠고기사태찜', '쇠갈비찜', '돼지고기 수육', '돼지갈비찜', '고등어조림', '건갈치조림', '갈치조림', '꽁치조림', '닭고추장조림', '닭조림', '코다리조림', '양미리조림', '삼치조림', '병어조림', '미트볼조림', '육회']
len(outlier75_5_10)

101

In [442]:
outlier90_7_10 = ['제육덮밥', '잡채밥', '쇠머리국밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '내장탕', '감자탕', '꽁치구이', '고등어석쇠구이', '고등어구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '콩국수', '쫄면', '잔치국수', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '간짜장', '채소떡볶이', '참치 김치볶음', '제육볶음', '쇠고기볶음', '햄채소볶음', '돼지고기채소볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '등심돈까스', '닭강정', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '달걀찜(우유)', '꼬막찜', '가오리찜', '달걀찜(새우젓)', '닭찜', '해물콩나물찜', '코다리찜', '족발', '아귀찜', '순대', '쇠고기수육', '쇠고기사태찜', '쇠갈비찜', '돼지고기 수육', '돼지갈비찜', '고등어조림', '건갈치조림', '갈치조림', '꽁치조림', '닭고추장조림', '닭조림', '코다리조림', '양미리조림', '삼치조림', '병어조림', '미트볼조림', '육회']
len(outlier90_7_10)

76

In [425]:
outlier100_7_10 = ['제육덮밥', '잡채밥', '쇠머리국밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '내장탕', '감자탕', '꽁치구이', '고등어석쇠구이', '고등어구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '쫄면', '잔치국수', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '채소떡볶이', '제육볶음', '쇠고기볶음', '햄채소볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '등심돈까스', '닭강정', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '달걀찜(우유)', '꼬막찜', '가오리찜', '달걀찜(새우젓)', '닭찜', '해물콩나물찜', '코다리찜', '족발', '아귀찜', '순대', '쇠고기수육', '쇠고기사태찜', '쇠갈비찜', '돼지고기 수육', '돼지갈비찜', '고등어조림', '건갈치조림', '갈치조림', '꽁치조림', '닭고추장조림', '닭조림', '코다리조림', '양미리조림', '삼치조림', '병어조림', '미트볼조림', '육회']
len(outlier100_7_10)

72

In [426]:
outlier110_7_10 = ['제육덮밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '내장탕', '꽁치구이', '고등어석쇠구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '회냉면', '쫄면', '잔치국수', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '채소떡볶이', '제육볶음', '쇠고기볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '달걀찜(우유)', '꼬막찜', '가오리찜', '달걀찜(새우젓)', '닭찜', '해물콩나물찜', '코다리찜', '족발', '아귀찜', '순대', '쇠고기수육', '쇠고기사태찜', '쇠갈비찜', '돼지고기 수육', '돼지갈비찜', '고등어조림', '건갈치조림', '갈치조림', '꽁치조림', '닭고추장조림', '닭조림', '코다리조림', '양미리조림', '삼치조림', '병어조림', '미트볼조림', '육회']
len(outlier110_7_10)

65

In [463]:
outlier125_8_10 = ['제육덮밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '꽁치구이', '고등어석쇠구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '오리고기구이', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '잔치국수', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '채소떡볶이', '쇠고기볶음', '닭볶음', '삼계탕', '닭곰탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '깐풍기', '새우탕수육', '탕수육', '치즈돈가스', '족발', '순대', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '고등어조림', '건갈치조림', '갈치조림', '꽁치조림', '닭고추장조림', '닭조림', '코다리조림', '양미리조림', '삼치조림', '병어조림', '미트볼조림', '육회']
len(outlier125_8_10)

51

In [393]:
outlier150_8_10 = ['제육덮밥', '돼지머리국밥', '장어탕', '오리탕', '부대찌개', '꽁치구이', '고등어석쇠구이', '간장양념닭다리구이', '닭구이', '돼지갈비구이', '치킨데리야끼', '쇠갈비구이', '삼겹살구이', '삼겹살고추장구이', '우동(일식)', '미트볼 토마토 스파게티', '메밀국수', '막국수', '닭볶음', '삼계탕', '순댓국', '갈비탕', '사골만두국', '떡만두국', '떡국', '새우탕수육', '족발', '순대', '쇠고기수육', '돼지고기 수육', '돼지갈비찜', '고등어조림', '건갈치조림', '갈치조림', '꽁치조림', '닭고추장조림', '닭조림', '코다리조림', '양미리조림', '삼치조림', '병어조림', '미트볼조림', '육회']
len(outlier150_8_10)

43

In [401]:
outlier200_9_10 = ['돼지머리국밥', '오리탕', '부대찌개', '우동(일식)', '삼계탕', '사골만두국', '떡만두국', '떡국', '족발', '돼지고기 수육', '돼지갈비찜', '육회']
len(outlier200_9_10)

12

In [467]:
replist

['하이라이스', '호박고추장찌개', '함박스테이크', '칼국수', '팔보채', '뼈해장국', '간장양념치킨', '달걀찜(우유)']

In [468]:
outlier = outlier125_8_10

In [469]:
df_copy2 = df_copy[df_copy['식품명'].isin(replist + outliers)]

In [470]:
df_copy2

,식품명,1회제공량,식품대분류,에너지(㎉),탄수화물(g),단백질(g),지방(g),총당류(g),나트륨(g)
7073,팔보채,220,볶음류,196.98,7.53,27.33,6.39,4.59,1.88399
7071,채소떡볶이,200,볶음류,515.29,114.11,6.24,3.76,3.94,0.53797
7053,쇠고기볶음,190,볶음류,543.39,5.40,33.24,43.20,8.16,0.56262
7018,하이라이스,360,밥류,401.93,76.84,14.47,4.08,0.00,0.88409
7002,제육덮밥,470,밥류,949.76,79.22,44.30,50.63,9.30,0.81984
7031,닭볶음,300,볶음류,558.83,23.80,41.93,32.88,2.32,1.01389
7164,육회,150,회류,195.06,12.18,14.90,9.64,7.93,0.62701
7209,고등어조림,100,조림류,141.01,4.10,11.25,8.84,3.39,0.38957
7208,건갈치조림,100,조림류,127.65,4.64,15.93,5.04,5.35,1.16912
7206,갈치조림,100,조림류,100.82,6.16,12.72,2.81,6.29,0.56755


In [471]:
df_copy2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 7073 to 282
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   식품명      59 non-null     object 
 1   1회제공량    59 non-null     int64  
 2   식품대분류    59 non-null     object 
 3   에너지(㎉)   59 non-null     float64
 4   탄수화물(g)  59 non-null     float64
 5   단백질(g)   59 non-null     float64
 6   지방(g)    59 non-null     float64
 7   총당류(g)   59 non-null     float64
 8   나트륨(g)   59 non-null     float64
dtypes: float64(6), int64(1), object(2)
memory usage: 4.6+ KB


In [472]:
df_copy2 = df_copy2.drop(['1회제공량', '식품대분류',"에너지(㎉)"], axis=1)

In [473]:
df_copy2['총합'] = df_copy2['탄수화물(g)'] + df_copy2['단백질(g)'] + df_copy2['지방(g)'] + df_copy2['총당류(g)'] + df_copy2['나트륨(g)']
df_copy2 = df_copy2[['식품명', '총합', '탄수화물(g)', '단백질(g)', '지방(g)', '총당류(g)', '나트륨(g)']]

In [474]:
df_copy2 = df_copy2.sort_values(by='총합', ascending=False)

In [475]:
df_copy2

,식품명,총합,탄수화물(g),단백질(g),지방(g),총당류(g),나트륨(g)
6956,우동(일식),285.24292,243.83,14.27,25.75,0.14,1.25292
6978,돼지머리국밥,253.31451,143.42,60.34,46.47,1.46,1.62451
6944,막국수,219.51715,143.37,34.81,9.63,29.28,2.42715
7002,제육덮밥,184.26984,79.22,44.30,50.63,9.30,0.81984
6859,삼계탕,178.61667,34.57,96.09,43.19,4.03,0.73667
6949,미트볼 토마토 스파게티,174.72914,88.08,36.70,44.41,4.45,1.08914
6805,돼지갈비구이,167.40165,10.43,84.72,55.61,14.43,2.21165
6803,닭구이,163.71309,26.52,89.10,45.54,0.00,2.55309
7276,오리탕,156.17356,36.46,56.99,39.37,21.45,1.90356
7296,돼지고기 수육,148.49128,7.05,59.65,74.46,7.21,0.12128


In [476]:
df_copy2.to_csv('df_copy5.csv', index=False)

# K-MEANS

In [198]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 7073 to 275
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   식품명      218 non-null    object 
 1   1회제공량    218 non-null    int64  
 2   식품대분류    218 non-null    object 
 3   에너지(㎉)   218 non-null    float64
 4   탄수화물(g)  218 non-null    float64
 5   단백질(g)   218 non-null    float64
 6   지방(g)    218 non-null    float64
 7   총당류(g)   218 non-null    float64
 8   나트륨(㎎)   218 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 17.0+ KB


In [202]:
# K-means 클러스터링
k = 20  # 군집 개수
kmeans = KMeans(n_clusters = k)

data = df_copy[["에너지(㎉)", "탄수화물(g)", "단백질(g)", "지방(g)", "총당류(g)", "나트륨(㎎)"]].values
item_names = df_copy["식품명"].values


data

array([[1.96980e+02, 7.53000e+00, 2.73300e+01, 6.39000e+00, 4.59000e+00,
        1.88399e+03],
       [5.15290e+02, 1.14110e+02, 6.24000e+00, 3.76000e+00, 3.94000e+00,
        5.37970e+02],
       [1.21880e+02, 7.12000e+00, 8.33000e+00, 6.68000e+00, 1.83000e+00,
        4.79740e+02],
       ...,
       [6.51750e+02, 3.42500e+01, 3.32800e+01, 4.24000e+01, 2.77000e+00,
        5.51550e+02],
       [6.53440e+02, 4.65600e+01, 2.43300e+01, 4.11000e+01, 1.96000e+00,
        7.88620e+02],
       [6.23830e+02, 3.84900e+01, 3.29900e+01, 3.75400e+01, 3.66000e+00,
        5.74080e+02]])

In [203]:
kmeans.fit(data)

KMeans(n_clusters=20)

In [204]:
# 각 데이터의 클러스터 할당 결과
labels = kmeans.labels_
clusters = {}
for i, label in enumerate(labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(df.iloc[i]['식품명'])

# 클러스터별 데이터 목록 출력
for cluster, data in clusters.items():
    print("클러스터 {}: {}".format(cluster, data))

클러스터 19: ['팔보채', '햄채소볶음', '닭발볶음', '선짓국(선지해장국)', '닭발구이', '쇠곱창구이', '달래된장찌개', '감자탕']
클러스터 10: ['채소떡볶이', '쇠고기볶음', '오징어덮밥', '꽁치구이', '회냉면', '콩국수', '생선모듬초밥', '새우볶음밥', '모듬회덮밥', '닭고기덮밥', '닭날개튀김', '안심돈가스', '등심돈가스']
클러스터 15: ['참치 김치볶음', '유산슬', '고등어조림', '갈치조림', '달걀찜(우유)', '호박고추장찌개', '청국장찌개', '순두부찌개', '달걀찜(새우젓)', '버섯전골', '해물콩나물찜', '코다리조림', '삼치조림', '두부쇠고기고추장찌개', '된장찌개', '돼지고기김치찌개', '동태찌개', '냉이된장찌개']
클러스터 5: ['죽순쇠고기볶음', '치즈김밥', '참치마요삼각김밥', '돼지곱창순대볶음', '함박스테이크', '삼치구이', '돼지불고기', '쇠고기볶음밥', '오리백숙', '미트볼조림', '후라이드치킨']
클러스터 3: ['제육볶음', '콩나물국밥', '참치볶음밥', '육회비빔밥', '갈비탕', '오리고기구이', '알밥', '칼국수', '쫄면', '짬뽕', '라면', '떡라면', '돌솥비빔밥', '김치볶음밥', '닭강정', '순대', '돼지갈비찜', '부대찌개', '내장탕']
클러스터 1: ['잡채', '오징어볶음', '쇠고기채소볶음', '라볶이', '떡볶이', '육회', '갈치구이', '가자미구이', '새우구이', '메밀국수', '쇠고기주먹밥', '김밥', '닭조림', '어묵김치찌개', '쇠고기 샤브샤브', '닭찜', '깐풍기', '두부애호박된장찌개', '꽁치김치찌개']
클러스터 4: ['하이라이스', '표고버섯볶음밥', '채소김밥', '참치김밥', '유부초밥', '돼지고기채소볶음', '닭고기대파꼬치구이', '참치머리구이', '쇠불고기', '수제비', '비빔냉면', '볶음우동', '고추김밥', '쇠고기김밥', '날치알김밥', '낙지덮밥', '김치김밥', '닭고추장조림', '